# Chapter 27: Computational Geometry

> *"Geometry is the foundation of all spatial reasoning. In computing, it enables us to model and manipulate the physical world."* — Anonymous

---

## 27.1 Introduction to Computational Geometry

**Computational geometry** is the study of algorithms and data structures for solving geometric problems. It has applications in computer graphics, robotics, geographic information systems (GIS), computer-aided design (CAD), and computer vision.

### 27.1.1 Why Computational Geometry Matters

```
┌─────────────────────────────────────────────────────────────────────┐
│                    IMPORTANCE OF COMPUTATIONAL GEOMETRY               │
├─────────────────────────────────────────────────────────────────────┤
│                                                                      │
│  1. COMPUTER GRAPHICS: Rendering, collision detection, mesh generation│
│  2. ROBOTICS: Path planning, obstacle avoidance, motion planning    │
│  3. GIS: Map overlay, routing, terrain analysis                      │
│  4. CAD: Design, simulation, manufacturing                           │
│  5. COMPUTER VISION: Shape recognition, object tracking               │
│  6. MACHINE LEARNING: Geometric feature extraction, clustering        │
│  7. BIOINFORMATICS: Protein structure analysis, molecular modeling    │
│                                                                      │
└─────────────────────────────────────────────────────────────────────┘
```

### 27.1.2 Basic Geometric Objects

- **Point:** A location in space (usually 2D or 3D) represented by coordinates (x, y).
- **Vector:** Direction and magnitude; can represent displacement.
- **Line:** Infinite set of points; defined by two points or a point and direction.
- **Line segment:** Finite portion of a line between two endpoints.
- **Polygon:** Closed chain of line segments; simple if no self-intersection.
- **Circle:** Set of points at a fixed distance (radius) from a center.

---

## 27.2 Geometric Primitives and Operations

### 27.2.1 Representing Points and Vectors

We'll use simple classes or just tuples. For clarity, we'll use classes with x, y.

```python
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __repr__(self):
        return f"({self.x}, {self.y})"
    
    def __sub__(self, other):
        return Vector(self.x - other.x, self.y - other.y)

class Vector:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def cross(self, other):
        return self.x * other.y - self.y * other.x
    
    def dot(self, other):
        return self.x * other.x + self.y * other.y
```

### 27.2.2 Orientation Test (Cross Product)

Given three points p, q, r, the orientation tells us whether r is to the left of, right of, or collinear with the directed line from p to q.

```
cross = (q.x - p.x)*(r.y - p.y) - (q.y - p.y)*(r.x - p.x)
```

- If cross > 0 → counter-clockwise (r is left of pq).
- If cross < 0 → clockwise (r is right of pq).
- If cross = 0 → collinear.

```python
def orientation(p, q, r):
    val = (q.x - p.x)*(r.y - p.y) - (q.y - p.y)*(r.x - p.x)
    if val == 0:
        return 0  # collinear
    return 1 if val > 0 else -1  # 1: CCW, -1: CW
```

### 27.2.3 On-Segment Test

Check if a point r lies on the segment pq (assuming collinear).

```python
def on_segment(p, q, r):
    return (min(p.x, q.x) <= r.x <= max(p.x, q.x) and
            min(p.y, q.y) <= r.y <= max(p.y, q.y))
```

---

## 27.3 Line Segment Intersection

Determine whether two line segments (p1,q1) and (p2,q2) intersect.

Algorithm: Use orientation tests.

- General case: segments intersect if orientations (p1,q1,p2) and (p1,q1,q2) are opposite and orientations (p2,q2,p1) and (p2,q2,q1) are opposite.
- Special collinear cases: check if projections overlap.

```python
def segments_intersect(p1, q1, p2, q2):
    o1 = orientation(p1, q1, p2)
    o2 = orientation(p1, q1, q2)
    o3 = orientation(p2, q2, p1)
    o4 = orientation(p2, q2, q1)
    
    if o1 != o2 and o3 != o4:
        return True
    if o1 == 0 and on_segment(p1, q1, p2):
        return True
    if o2 == 0 and on_segment(p1, q1, q2):
        return True
    if o3 == 0 and on_segment(p2, q2, p1):
        return True
    if o4 == 0 and on_segment(p2, q2, q1):
        return True
    return False
```

---

## 27.4 Convex Hull Algorithms

The **convex hull** of a set of points is the smallest convex polygon that contains all points. It is a fundamental structure in computational geometry.

### 27.4.1 Graham Scan

**Steps:**
1. Find the point with the lowest y (and leftmost if tie). This is the starting point.
2. Sort all other points by polar angle relative to the start.
3. Initialize stack with start.
4. For each point in sorted order, while the last three points make a non-left turn (i.e., orientation of second-last, last, current is clockwise or collinear), pop from stack.
5. Push current point.

```python
def graham_scan(points):
    # Find bottom-most point (lowest y, leftmost if tie)
    start = min(points, key=lambda p: (p.y, p.x))
    
    # Sort points by polar angle relative to start
    def polar_angle(p):
        return math.atan2(p.y - start.y, p.x - start.x)
    
    sorted_points = sorted([p for p in points if p != start], key=polar_angle)
    
    stack = [start]
    for p in sorted_points:
        while len(stack) >= 2 and orientation(stack[-2], stack[-1], p) <= 0:
            stack.pop()
        stack.append(p)
    return stack
```

**Time:** O(n log n) for sorting.

### 27.4.2 Jarvis March (Gift Wrapping)

**Steps:**
1. Find the leftmost point.
2. Repeatedly find the point with the smallest polar angle relative to the last hull point.
3. Stop when we return to the start.

```python
def jarvis_march(points):
    if len(points) < 3:
        return points
    # Find leftmost point
    leftmost = min(points, key=lambda p: p.x)
    hull = []
    p = leftmost
    while True:
        hull.append(p)
        q = points[0]  # next candidate
        for r in points:
            if r == p:
                continue
            # Find most counterclockwise point relative to p
            if q == p or orientation(p, q, r) == 1:
                q = r
        p = q
        if p == leftmost:
            break
    return hull
```

**Time:** O(nh) where h is number of hull points. Worst-case O(n²).

### 27.4.3 Monotone Chain (Andrew's Algorithm)

**Steps:**
1. Sort points by x (then y).
2. Build lower hull: iterate sorted points, maintain stack, pop while last three make non-left turn.
3. Build upper hull similarly, iterating from right to left.
4. Combine lower and upper (excluding duplicates of endpoints).

```python
def monotone_chain(points):
    points = sorted(points, key=lambda p: (p.x, p.y))
    if len(points) <= 1:
        return points
    
    # Lower hull
    lower = []
    for p in points:
        while len(lower) >= 2 and orientation(lower[-2], lower[-1], p) <= 0:
            lower.pop()
        lower.append(p)
    
    # Upper hull
    upper = []
    for p in reversed(points):
        while len(upper) >= 2 and orientation(upper[-2], upper[-1], p) <= 0:
            upper.pop()
        upper.append(p)
    
    # Concatenate lower and upper, removing last point of each because it's repeated
    return lower[:-1] + upper[:-1]
```

**Time:** O(n log n) for sorting.

---

## 27.5 Closest Pair of Points (Divide and Conquer)

This problem is already covered in Chapter 21 (Divide and Conquer). However, we revisit briefly with a more geometric focus.

**Algorithm:**
1. Sort points by x-coordinate.
2. Recursively find closest pair in left and right halves. Let δ = min(left, right).
3. Consider points within a vertical strip of width 2δ around the midline. Sort these by y-coordinate.
4. For each point in the strip, check distances to next 7 points (or up to 15 depending on implementation) to find any pair closer than δ.
5. Return the minimum.

```python
import math

def closest_pair(points):
    points_x = sorted(points, key=lambda p: p.x)
    return _closest_pair_rec(points_x)

def _closest_pair_rec(px):
    n = len(px)
    if n <= 3:
        return brute_force(px)
    mid = n // 2
    mid_x = px[mid].x
    left = px[:mid]
    right = px[mid:]
    d1, pair1 = _closest_pair_rec(left)
    d2, pair2 = _closest_pair_rec(right)
    d = min(d1, d2)
    best_pair = pair1 if d1 < d2 else pair2
    # Strip
    strip = [p for p in px if abs(p.x - mid_x) < d]
    strip.sort(key=lambda p: p.y)
    for i in range(len(strip)):
        for j in range(i+1, len(strip)):
            if strip[j].y - strip[i].y >= d:
                break
            dist = math.hypot(strip[i].x - strip[j].x, strip[i].y - strip[j].y)
            if dist < d:
                d = dist
                best_pair = (strip[i], strip[j])
    return d, best_pair

def brute_force(points):
    min_dist = float('inf')
    best = None
    n = len(points)
    for i in range(n):
        for j in range(i+1, n):
            dist = math.hypot(points[i].x - points[j].x, points[i].y - points[j].y)
            if dist < min_dist:
                min_dist = dist
                best = (points[i], points[j])
    return min_dist, best
```

**Time:** O(n log n)

---

## 27.6 Point in Polygon (Ray Casting)

Determine whether a point lies inside a simple polygon. One common method is ray casting: count intersections of a horizontal ray to the right with polygon edges. Odd count → inside.

```python
def point_in_polygon(point, polygon):
    # polygon: list of Points in order (closed, last may equal first)
    cnt = 0
    n = len(polygon)
    for i in range(n):
        p1 = polygon[i]
        p2 = polygon[(i+1) % n]
        # Check if edge straddles horizontal line at point.y
        if ((p1.y > point.y) != (p2.y > point.y)):
            # Compute x-intersection of edge with horizontal line
            x_int = p1.x + (point.y - p1.y) * (p2.x - p1.x) / (p2.y - p1.y)
            if x_int > point.x:
                cnt += 1
    return cnt % 2 == 1
```

**Time:** O(n)

---

## 27.7 Area of a Polygon

Given a simple polygon (list of vertices in order), compute its signed area using the shoelace formula.

```python
def polygon_area(polygon):
    area = 0
    n = len(polygon)
    for i in range(n):
        p1 = polygon[i]
        p2 = polygon[(i+1) % n]
        area += p1.x * p2.y - p2.x * p1.y
    return abs(area) / 2.0
```

**Signed area** positive if vertices are in counter-clockwise order.

---

## 27.8 Sweep Line Algorithms

Sweep line algorithms process geometric events in order (e.g., by x-coordinate) and maintain a data structure to answer queries. They are used for line segment intersection, Fortune's algorithm for Voronoi diagrams, etc.

### 27.8.1 Line Segment Intersection (Bentley-Ottmann)

Finds all intersections among a set of line segments in O((n + k) log n) time, where k is number of intersections. It sweeps a vertical line, maintaining active segments ordered by y, and handles events: segment start, segment end, and intersection points.

**Key ideas:**
- Event queue (priority queue) sorted by x.
- Status structure (balanced BST) storing active segments sorted by current y.
- When two segments intersect, generate an intersection event.

Implementation is complex; we outline steps.

---

## 27.9 Voronoi Diagrams and Delaunay Triangulation

### 27.9.1 Voronoi Diagram

For a set of points (sites), the Voronoi diagram partitions the plane into regions where each region contains points closer to that site than any other. Edges are equidistant between two sites, vertices equidistant among three sites.

**Fortune's algorithm** sweeps a line and uses a beach line of parabolas to construct the diagram in O(n log n). It's complex but widely used.

### 27.9.2 Delaunay Triangulation

The Delaunay triangulation of a set of points is the dual of the Voronoi diagram: connect sites whose Voronoi cells share an edge. It maximizes the minimum angle among triangles, avoiding skinny triangles. Used in mesh generation, interpolation.

**Properties:**
- No point lies inside the circumcircle of any triangle.
- Equivalent to the convex hull in 3D (lift points to paraboloid).

**Algorithms:**
- Bowyer-Watson (incremental) O(n²) or O(n log n) with spatial indexing.
- Divide and conquer (O(n log n)).

---

## 27.10 Applications

- **Collision detection:** Check if convex polygons intersect (separating axis theorem).
- **Robot path planning:** Configuration space obstacles, visibility graphs.
- **GIS:** Map overlay, shortest path with obstacles.
- **Computer graphics:** Ray tracing, hidden surface removal.
- **Mesh generation:** Delaunay triangulation for finite element methods.
- **Pattern recognition:** Shape matching, convex hull for gesture recognition.

---

## 27.11 Summary

```
┌────────────────────────────┬────────────────────────────────────────┐
│ Problem                    │ Algorithms / Techniques                │
├────────────────────────────┼────────────────────────────────────────┤
│ Convex Hull                │ Graham Scan (O(n log n)), Jarvis (O(nh)), │
│                            │ Monotone Chain (O(n log n))              │
│ Closest Pair               │ Divide and Conquer (O(n log n))        │
│ Line Intersection          │ Orientation + on-segment test          │
│ Point in Polygon           │ Ray casting (O(n))                     │
│ Polygon Area               │ Shoelace formula (O(n))                 │
│ Line Segment Intersections │ Sweep line (Bentley-Ottmann) O((n+k) log n)│
│ Voronoi Diagram            │ Fortune's algorithm (O(n log n))       │
│ Delaunay Triangulation     │ Bowyer-Watson, Divide & Conquer        │
└────────────────────────────┴────────────────────────────────────────┘
```

---

## 27.12 Practice Problems

### Easy/Medium
1. **Convex Hull** (LeetCode 587) – Erect the Fence (monotone chain).
2. **Largest Perimeter Triangle** (LeetCode 976) – sort, check triangle inequality.
3. **Minimum Time to Visit a Point** (LeetCode 1266) – Chebyshev distance.
4. **Valid Triangle Number** (LeetCode 611) – two-pointer.

### Medium
5. **Max Points on a Line** (LeetCode 149) – slope counting.
6. **Rectangle Overlap** (LeetCode 836) – axis-aligned rectangles.
7. **Line Reflection** (LeetCode 356) – check symmetry.
8. **Brick Wall** (LeetCode 554) – not exactly geometry but uses intervals.

### Hard
9. **Closest Pair of Points** (not on LC but classic) – implement.
10. **Line Segment Intersections** (LeetCode? Not directly, but can be used in problems like "Number of Intersections in a Grid").
11. **Circle and Rectangle Overlapping** (LeetCode 1401).
12. **Minimum Area Rectangle** (LeetCode 939) – sweep or hash.
13. **Minimum Area Rectangle II** (LeetCode 963) – more complex.
14. **The Skyline Problem** (LeetCode 218) – sweep line.

---

## 27.13 Further Reading

1. **"Computational Geometry: Algorithms and Applications"** by de Berg, Cheong, van Kreveld, Overmars – The classic textbook.
2. **"Geometric Tools for Computer Graphics"** by Schneider & Eberly.
3. **"The Algorithm Design Manual"** by Steven Skiena – Chapter 17 (Computational Geometry).
4. **Original Papers**:
   - Graham, R. L. (1972) – "An efficient algorithm for determining the convex hull of a finite planar set"
   - Jarvis, R. A. (1973) – "On the identification of the convex hull of a finite set of points in the plane"
   - Fortune, S. (1987) – "A sweepline algorithm for Voronoi diagrams"
   - Bentley, J. L., & Ottmann, T. A. (1979) – "Algorithms for reporting and counting geometric intersections"

---

> **Coming in Chapter 28**: **Disjoint Set Union (Union-Find)** – We'll explore this elegant data structure for managing partitions and its applications in connectivity.

---

**End of Chapter 27**